<a href="https://colab.research.google.com/github/deanzedd/Machine-Learning/blob/main/KNN_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# khoảng cách điểm dữ liệu

code mẫu phần tính khoảng cách các điểm dữ liệu

In [ ]:
from __future__ import print_function
import numpy as np
from time import time #for comparing running time
d, N=1000, 10000 #dimension and number of training points
X=np.random.randn(N,d) #N d-dimensional points
z=np.random.randn(d) #d-dimensional vector


In [ ]:
#naively compute square distance between two vector(point to point)
def dist_pp(z,x) :
  d = z - x.reshape(z.shape) # force x and z to have the same dims
  return np.sum(d*d)

## ps = point to set
#from one point to each point in a set, naive
def dist_ps_naive(z,X) :
  N = X.shape[0]
  res = np.zeros((1,N))
  for i in range(N) :
    res[0][i] = dist_pp(z,X[i])
  return res

# from one point to each point in a set, fast
def dist_ps_fast(z,X) :
  X2 = np.sum(X*X,1) # square of l2 norm of each X[i], can be precomputed
  z2 = np.sum(z*z) # square of l2 norm of z
  return X2 + z2 - 2*X.dot(z) # z2 can be ignored

t1 = time()
D1 = dist_ps_naive(z,X)
print('naive point2set, running time:', time() - t1, 's')

t1 = time()
D2 = dist_ps_fast(z,X)
print('fast point2set , running time:', time() - t1, 's')
print('Result difference:', np.linalg.norm(D1 - D2))



naive point2set, running time: 0.16475605964660645 s
fast point2set , running time: 0.06168937683105469 s
Result difference: 2.3863406147372906e-11


In [ ]:
Z=np.random.randn(100,d) #100 d-dimesional points
#ss= set to set
#from each point in one set to each point in another set, half fast
def dist_ss_0(Z,X) :
  M,N = Z.shape[0],X.shape[0]
  res = np.zeros((M,N))
  for i in range(M) :
    res[i] = dist_ps_fast(Z[i],X)
  return res
print(Z.shape) #(100,1000=d)
print(X.shape[0]) #10000=N

#from each point in one set to each point in another set, fast
def dist_ss_fast(Z,X) :
  X2 = np.sum(X*X,1) # square of l2 norm of each row of X
  Z2 = np.sum(Z*Z,1) # square of l
  return Z2.reshape(-1,1) + X2.reshape(1,-1) - 2*Z.dot(X.T)

t1 = time()
D3 = dist_ss_0(Z,X)
print('half fast set2set running time:', time() - t1, 's')
t1 = time()
D4 = dist_ss_fast(Z,X)
print('fast set2set running time:', time() - t1, 's')
print('Result difference:', np.linalg.norm(D3 - D4))

(100, 1000)
10000
half fast set2set running time: 6.827006101608276 s
fast set2set running time: 0.12366271018981934 s
Result difference: 1.008814233156002e-10


# IRis flower data set

In [ ]:
from __future__ import print_function
import numpy as np
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split #for splitting data
from sklearn.metrics import accuracy_score #for evaluating results

iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print('Labels:', np.unique(iris_y))

# split train and test
np.random.seed(7) #7 thêm vào để đảm bảo kết quả chạy(7 có thể thay bằng số bất kì 32 bit)
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=130)

print('Train size:', X_train.shape[0], ', test size:', X_test.shape[0])


Labels: [0 1 2]
Train size: 20 , test size: 130


*result of 1NN*

In [ ]:
#result of 1NN
model = neighbors.KNeighborsClassifier(n_neighbors = 1, p = 2) #K=1,p=2 chính là l2 norm(p=1 là l1 norm)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 1NN: 92.31 %


*result of 7NN*

In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors= 7, p = 2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 7NN with major voting: 93.85 %


*đánh trọng số điểm lân cận*

điểm càng gần thì càng tốt-> trọng số cao
weights = ’distance’

weights là ’uniform’ (default)


In [ ]:
model = neighbors.KNeighborsClassifier(n_neighbors= 7, p=2, weights='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN (1/distance weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 7NN (1/distance weights): 94.62 %


*KNN tự định nghĩa*

In [ ]:
def myweight(distances) :
  sigma2 = .4 # we can change this number
  return np.exp(-distances**2/sigma2)

model = neighbors.KNeighborsClassifier(n_neighbors= 7, p=2, weights=myweight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy of 7NN (customized weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))
#

Accuracy of 7NN (customized weights): 95.38 %


# TỔNG KẾT

ƯU ĐIỂM
- độ phức tạp thấp(do tính toán nhiều ở pha huấn luyện)
- việc dự đoán kết quả đơn giản khi xác định được các lân cận
- KNN ko cần giả sử phân phối của từng nhãn

NHƯỢC ĐIỂM
- KNN nhạy cảm khi K nhỏ
- KNN phần lớn tính toán ở pha kiểm tra.
trong đó phần lớn tính toán ở các điểm dữ liệu huấn luyện nên sẽ tốn nhiều thời gian